In [6]:
!pip install nltk


DEPRECATION: Loading egg at c:\users\daneil\appdata\local\programs\python\python311\lib\site-packages\magicformulabr-0.1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem.porter import PorterStemmer
import ast


In [8]:

# Carregar os dados
credits_df = pd.read_csv('credits.csv')
movies_df = pd.read_csv('movies.csv')
movies_df = movies_df.merge(credits_df, on='title')

# Selecionar colunas relevantes
movies_df = movies_df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew', 'vote_average', 'vote_count']]

# Tratar valores nulos
movies_df.dropna(inplace=True)

# Funções para converter gêneros, palavras-chave, elenco e diretores em listas de nomes
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

def fetch_directors(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L



In [9]:
# Aplicar as funções de conversão
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)
movies_df['cast'] = movies_df['cast'].apply(convert3)
movies_df['crew'] = movies_df['crew'].apply(fetch_directors)

# Tratar espaços em branco
movies_df['overview'] = movies_df['overview'].apply(lambda x: x.split())
movies_df['genres'] = movies_df['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x: [i.replace(" ", "") for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

# Criar a coluna 'tags'
movies_df['tags'] = movies_df['overview'] + movies_df['genres'] + movies_df['keywords'] + movies_df['cast'] + movies_df['crew']

# Criar um DataFrame final com as colunas necessárias
df = movies_df[['movie_id', 'title', 'tags', 'genres', 'vote_average', 'vote_count']]

# Converter a lista de tags em uma string
df['tags'] = df['tags'].apply(lambda x: ' '.join(x))

# Converter as tags para minúsculas
df['tags'] = df['tags'].apply(lambda x: x.lower())




C:\Users\daneil\AppData\Local\Temp\ipykernel_13468\1041280938.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: ' '.join(x))
C:\Users\daneil\AppData\Local\Temp\ipykernel_13468\1041280938.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: x.lower())


In [10]:
# Inicializar CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(df['tags']).toarray()

# Inicializar PorterStemmer
ps = PorterStemmer()

# Função para aplicar stemming
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

# Aplicar stemming
df['tags'] = df['tags'].apply(stem)

# Calcular a similaridade coseno
similarity = cosine_similarity(vectors)



C:\Users\daneil\AppData\Local\Temp\ipykernel_13468\2005820961.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(stem)


In [11]:
# Funções de recomendação
def recommend(movie, n_recommendations=5):
    movie_index = df[df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:n_recommendations+1]
    recommendations = []
    for i in movies_list:
        title = df.iloc[i[0]].title
        score = df.iloc[i[0]].vote_average
        count = df.iloc[i[0]].vote_count
        recommendations.append((title, score, count))
    return recommendations

def recomendagenero(movie, genero, n_recommendations=5):
    listamostra = []
    conta = 0
    movie_index = df[df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:]
    for i in movies_list:
        if genero in df.iloc[i[0]].genres:
            title = df.iloc[i[0]].title
            score = df.iloc[i[0]].vote_average
            count = df.iloc[i[0]].vote_count
            listamostra.append((title, score, count))
            conta += 1
            if conta == n_recommendations:
                break
    return listamostra



In [12]:
# Testar recomendações
recommendations = recommend('Avatar', 5)
print("Recomendações baseadas em 'Avatar':")
for rec in recommendations:
    print(f"{rec[0]} - Nota: {rec[1]} - Votos: {rec[2]}")

genero = 'Action'  
nome = 'Avatar'  
genero_recommendations = recomendagenero(nome, genero, 5)
print(f"\nRecomendações no gênero '{genero}' baseadas em '{nome}':")
for rec in genero_recommendations:
    print(f"{rec[0]} - Nota: {rec[1]} - Votos: {rec[2]}")

Recomendações baseadas em 'Avatar':
Titan A.E. - Nota: 6.3 - Votos: 313
Independence Day - Nota: 6.7 - Votos: 3260
Small Soldiers - Nota: 6.2 - Votos: 511
Aliens vs Predator: Requiem - Nota: 4.9 - Votos: 740
Battle: Los Angeles - Nota: 5.5 - Votos: 1448

Recomendações no gênero 'Action' baseadas em 'Avatar':
Titan A.E. - Nota: 6.3 - Votos: 313
Independence Day - Nota: 6.7 - Votos: 3260
Small Soldiers - Nota: 6.2 - Votos: 511
Aliens vs Predator: Requiem - Nota: 4.9 - Votos: 740
Battle: Los Angeles - Nota: 5.5 - Votos: 1448
